### 행정안전부 보도자료 데이터 수집
- 정적페이지 데이터 수집 : html : bs, css-selecter 이용
- post방식 데이터가져옴 (html문자열)
- 여러개의 페이지의 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 1. 웹서비스 분석 : url

In [8]:
page= 3
url = f"https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": page,
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
}

In [3]:
# 2. request(url) > response : html

In [9]:
response = requests.post(url,params)
response

<Response [200]>

In [10]:
response.text[:500] # html 포멧의 데이터 -> beautifulSoup, css-selecter 써야겠구나

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<title>\r\n\r\n행정안전부&gt;  뉴스·소식&gt; 보도자료&gt; 보도자료</title>\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<meta charset="utf-8" />\r\n<meta name="viewport" content="width=device-width,maximum-scale=1,minimum-scale=1,user-scalable=no" />\r\n<meta name="author" content="행정안전부" />\r\n<meta name="subject" content="행정안전부" />\r\n<meta name="keywords" content="행정안전부, 행안부, 정부혁신, 지방자치, 생활자치, 규제개혁, 지역경제 활성화, 국민생활 불편해소, 지방재정 개혁, 행정관리, 디지털 정부, 복지서비스, 복지사각지대, 지방공기업 경쟁'

In [4]:
# 3. html(str) > bs_obj > bs_obj.select(css-selecter) > text > DataFrame

In [12]:
dom = BeautifulSoup(response.text, "html.parser")

In [14]:
# 게시글 리스트 데이터 선택 : 10개
elements = dom.select(f"#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
len(elements)

10

In [27]:
# 각 데이터에서 필요한 정보 수집
element = elements[0]

data = {
    "no": element.select("td")[0].text.strip(), #앞뒤 공백제거 strip
    "title":element.select("td")[1].text.strip(),
    "writer":element.select("td")[3].text.strip(),
    "date":element.select("td")[4].text.strip(),
    "pv":element.select("td")[5].text.strip(),
    "link":"https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"), # a태그 선택
}
data

{'no': '12931',
 'title': '사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다',
 'writer': '공공지능정책과',
 'date': '2022.02.15.',
 'pv': '812',
 'link': 'https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=ZECKsf+l8hUneyzWVmtUwOP9.node40?bbsId=BBSMSTR_000000000008&nttId=90455'}

In [28]:
# df만들기 [{row1}, {row2}]

In [30]:
# 10개의 데이터 수집
datas = []
for element in elements:
    datas.append({
        "no": element.select("td")[0].text.strip(), #앞뒤 공백제거 strip
        "title":element.select("td")[1].text.strip(),
        "writer":element.select("td")[3].text.strip(),
        "date":element.select("td")[4].text.strip(),
        "pv":element.select("td")[5].text.strip(),
        "link":"https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"),
    })
datas

df = pd.DataFrame(datas)
df.tail(3)

,no,title,writer,date,pv,link
7,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,주민과,2022.02.10.,1760,https://www.mois.go.kr/frt/bbs/type010/commonS...
8,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1042,https://www.mois.go.kr/frt/bbs/type010/commonS...
9,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1120,https://www.mois.go.kr/frt/bbs/type010/commonS...


In [5]:
# 4. function : params : page

In [32]:
def mois(page):
    url = f"https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
    params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": page,
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
    }
    response = requests.post(url,params)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(f"#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
    datas = []
    for element in elements:
        datas.append({
            "no": element.select("td")[0].text.strip(), #앞뒤 공백제거 strip
            "title":element.select("td")[1].text.strip(),
            "writer":element.select("td")[3].text.strip(),
            "date":element.select("td")[4].text.strip(),
            "pv":element.select("td")[5].text.strip(),
            "link":"https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"),
        })
        
    return pd.DataFrame(datas)

In [33]:
mois(2)

,no,title,writer,date,pv,link
0,12941,코로나19 감염병 확산 대비 사회필수 기능 유지를 위한 기능연속성계획(BCP) 수립,예방안전과,2022.02.18.,1245,https://www.mois.go.kr/frt/bbs/type010/commonS...
1,12940,2022년도 정부청사 공무직 인사제도 이렇게 달라집니다,노사후생과,2022.02.17.,756,https://www.mois.go.kr/frt/bbs/type010/commonS...
2,12939,"위급상황, 소화기·완강기·자동심장충격기 사용은 이렇게!",예방안전과,2022.02.17.,524,https://www.mois.go.kr/frt/bbs/type010/commonS...
3,12938,"행정안전부 장관, 경북 영덕군 산불 관련 긴급지시",환경재난대응과,2022.02.16.,575,https://www.mois.go.kr/frt/bbs/type010/commonS...
4,12937,정부세종청사 관·군·경 통합방위태세 확립 훈련 실시,청사보안기획과,2022.02.16.,500,https://www.mois.go.kr/frt/bbs/type010/commonS...
5,12936,"공공부문 클라우드 정책, 공공·민간이 함께 만든다",디지털자원정책과,2022.02.16.,745,https://www.mois.go.kr/frt/bbs/type010/commonS...
6,12935,"경남청사관리소, 공무직 근로자와 노사화합의 첫 길을 열다",경남청사관리소,2022.02.16.,456,https://www.mois.go.kr/frt/bbs/type010/commonS...
7,12934,제20대 대통령선거 안전하고 공정하게 실시,선거의회자치법규과,2022.02.15.,655,https://www.mois.go.kr/frt/bbs/type010/commonS...
8,12933,"김부겸 국무총리, 제20대 대통령 선거 대비 「공명선거 관계장관회의」 주재",선거의회자치법규과,2022.02.15.,511,https://www.mois.go.kr/frt/bbs/type010/commonS...
9,12932,"정부청사, 청경 24시간 근무 폐지하는 단체협약 체결",청사보안기획과,2022.02.15.,575,https://www.mois.go.kr/frt/bbs/type010/commonS...


In [6]:
# 5. 여러 페이지 데이터 수집

In [35]:
# pandas의 concat 사용
dfs = []
for page in range(1,4):
    print(page, end="")
    dfs.append(mois(page))

123

In [39]:
dfs[0]
result_df =pd.concat(dfs, ignore_index=True)
result_df.tail(3)

,no,title,writer,date,pv,link
27,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,주민과,2022.02.10.,1762,https://www.mois.go.kr/frt/bbs/type010/commonS...
28,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1044,https://www.mois.go.kr/frt/bbs/type010/commonS...
29,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1122,https://www.mois.go.kr/frt/bbs/type010/commonS...
